In [25]:
!pip install datasets

In [27]:
from datasets import load_dataset

In [29]:
ds = load_dataset("malteos/wikinews", "en")

In [30]:
ds = ds["2015_q2_05"]

In [31]:
from datasets import load_dataset

def label_dataset(dataset):
    def has_common_categories(categories1, categories2, min_common=3):
        if not categories1 or not categories2:
            return False

        common_categories = set(categories1) & set(categories2)
        return len(common_categories) >= min_common

    labels = []
    matched_pairs = []

    for i, data_point in enumerate(dataset):
        is_labeled = False
        for j, compare_point in enumerate(dataset):
            if i != j:
                if has_common_categories(data_point['categories'], compare_point['categories']):
                    labels.append(1)
                    matched_pairs.append((i, j,
                        data_point['title'],
                        compare_point['title'],
                        list(set(data_point['categories']) & set(compare_point['categories']))
                    ))
                    is_labeled = True
                    break

        if not is_labeled:
            labels.append(0)

    # Add labels to the dataset
    dataset = dataset.add_column('label', labels)

    return dataset, matched_pairs


In [35]:
# Label the dataset
labeled_dataset, pairs = label_dataset(ds)

# Print matched pairs
print("Matched Pairs:")
for pair in pairs[:20]:  # Limit to first 20 pairs to avoid overwhelming output
    print(f"Rows {pair[0]} and {pair[1]} matched:")
    print(f"Title 1: {pair[2]}")
    print(f"Title 2: {pair[3]}")
    print(f"Common Categories: {pair[4]}")
    print("---")

# Optional: Save the labeled dataset
labeled_dataset.save_to_disk('labeled_wikinews_dataset')
print(labeled_dataset)


Matched Pairs:
Rows 0 and 26 matched:
Title 1: Two Australians executed in Indonesia over 2005 drug smuggling case
Title 2: Joe Hockey agrees to lobby states to ditch tampon tax
Common Categories: ['Tony Abbott', 'Oceania', 'Australia']
---
Rows 1 and 12 matched:
Title 1: Indiana Governor signs needle exchange program
Title 2: Over 900 asylum seekers rescued off Indonesian coast
Common Categories: ['United States', 'Politics and conflicts', 'North America']
---
Rows 2 and 11 matched:
Title 1: Stand by me: Music legend Ben E King dies at 76
Title 2: Blues musician B.B. King dies aged 89
Common Categories: ['United States', 'North America', 'Music']
---
Rows 3 and 8 matched:
Title 1: Royal Birth: Duchess of Cambridge gives birth to baby girl
Title 2: Conservatives win majority in 2015 UK general election
Common Categories: ['United Kingdom', 'David Cameron', 'Europe']
---
Rows 4 and 14 matched:
Title 1: German journalists detained in Qatar while investigating controversial 2022 FIFA Worl

Saving the dataset (0/1 shards):   0%|          | 0/35 [00:00<?, ? examples/s]

Dataset({
    features: ['language', 'wiki_page_id', 'wiki_revision_id', 'revision_timestamp', 'revision_year', 'revision_month', 'article_timestamp', 'article_year', 'article_month', 'url', 'title', 'raw_text', 'cleaned_text', 'categories', 'sources', 'dump', 'label'],
    num_rows: 35
})
